# Training the CNN model from scratch

This notebook is created to train the model with the CNN architecture from scratch. When the model is trained, prediction is done on the given test images, and the submission file is saved to 'submission.csv'. A post-processing step is also implemented, and the result after this is done is saved in 'submission_pp.csv'.

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from PIL import Image
import matplotlib.image as mpimg

import tensorflow.python.platform
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from sklearn.utils import class_weight, shuffle

from mask_to_submission import masks_to_submission
from cnn_pred import get_prediction_with_overlay_context, get_predictionimage_context, get_pred_postprocessed, make_img_overlay
from data_extraction import load_data_context
from cnn_model import create_model_cnn


NUM_CHANNELS = 3 # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 100
TESTING_SIZE = 50
VALIDATION_SIZE = 10  # Size of the validation set.

BATCH_SIZE = 16 
NUM_EPOCHS = 100
MAX_AUG = 2
IMG_PATCH_SIZE = 16
CONTEXT_SIZE = 16
NOISE_LEVEL = 0.006


'''Image paths'''
data_dir = '../data/'
train_data_filename = data_dir + 'training/images/'
train_labels_filename = data_dir + 'training/groundtruth/' 
test_data_filename = data_dir + 'test_set_images'


'''Path for storing the augmented training images'''
aug_img_path = data_dir +'training/augmented/images'
aug_gt_path = data_dir + 'training/augmented/groundtruth'


'''Path to store best weights and the submission file, and the predicted images'''
weight_path = '../weights/'
weight_filename = 'chechpoint.weights.cnn.hdf5'
submission_path = '../submission.csv'
pred_test_path = '../predictions_test/'


'''Path to store results after post-processing'''
postprocess_path = '../predictions_test_pp/'
pp_submission_path = '../submission_pp.csv'

Using TensorFlow backend.


### Loading data

In [2]:
x_train, y_train, x_test, x_val, y_val = load_data_context(train_data_filename, train_labels_filename,
          test_data_filename, TRAINING_SIZE, VALIDATION_SIZE, IMG_PATCH_SIZE, CONTEXT_SIZE, TESTING_SIZE,
          saltpepper = NOISE_LEVEL, augment=True, MAX_AUG=MAX_AUG, augImgDir=aug_img_path , data_dir=data_dir, groundTruthDir =aug_gt_path, newaugment=True)

Augmenting training images...
Directory  ../data/training/augmented/images  already exists, overwritten
Directory  ../data/training/augmented/groundtruth  already exists, overwritten
Loading test images

Train data shape:  (12500, 48, 48, 3)
Train labels shape:  (12500, 2)
Test data shape:  (14440, 48, 48, 3)


### Shuffle the training data

In [3]:
ind_list = [i for i in range(y_train.shape[0])]
shuffle(ind_list)
x_train  = x_train[ind_list, :,:,:]
y_train = y_train[ind_list,:]

### Computing class weights

In [4]:
classes = np.array([0,1])
class_weights = class_weight.compute_class_weight('balanced',classes,y_train[:,1])

print('Class weights: ',class_weights) 

Class weights:  [0.70185289 1.73852573]


### Loading model

In [5]:
img_rows = x_train[0].shape[1]
img_cols = img_rows
input_shape = (img_rows, img_cols, NUM_CHANNELS) 

model = create_model_cnn(input_shape, NUM_LABELS)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 252)       290556    
__________

### Compiling model

In [6]:
model.compile(loss=categorical_crossentropy,
          optimizer=Adam(),
          metrics=['accuracy'])

### Defining chechpoints to ensure the best weights are stored

In [7]:
if not os.path.isdir(weight_path):
    os.mkdir(weight_path)

checkpoint = ModelCheckpoint(weight_path+weight_filename, monitor='val_acc', 
                             verbose=1, save_best_only=True, mode='auto')
callbacks_list = [checkpoint]

### Training the model

In [8]:
model.fit(x_train, y_train,
          validation_data=(x_val, y_val),
          batch_size=BATCH_SIZE,
          epochs=NUM_EPOCHS,
          shuffle = True,
          verbose=1,
          class_weight = class_weights,
          callbacks = callbacks_list)

Train on 12500 samples, validate on 1875 samples
Epoch 1/1
12500/12500 [==============================] - 189s 15ms/step - loss: 0.4900 - acc: 0.7528 - val_loss: 0.3638 - val_acc: 0.8384

Epoch 00001: val_acc improved from -inf to 0.83840, saving model to ../weights/chechpoint.weights.cnn.hdf5


### Load the best weights from the training

In [9]:
model.load_weights(weight_path+weight_filename)

### Predicting on the test images, and creating overlay and groundtruth images

In [10]:
print('Creating submission...')
if not os.path.isdir(pred_test_path):
    os.mkdir(pred_test_path)
    
filenames = []
for i in range(1,TESTING_SIZE+1):
    if (i%np.floor(TESTING_SIZE/10) == 0):
        print(str(int(np.floor(i/np.floor(TESTING_SIZE/10))*10)), '% done')
    
    oimg = get_prediction_with_overlay_context(test_data_filename, i, 'test', model, IMG_PATCH_SIZE, CONTEXT_SIZE, PIXEL_DEPTH)
    oimg.save(pred_test_path + "overlay_" + str(i) + ".png")

    gt_filename = pred_test_path + "gt_pred_" + str(i) + ".png"
    imgpred = get_predictionimage_context(test_data_filename, i, 'test', model, IMG_PATCH_SIZE, CONTEXT_SIZE, PIXEL_DEPTH)
    filenames.append(gt_filename)
    imgpred.save(gt_filename)



Creating submission...
10 % done
20 % done
30 % done
40 % done
50 % done
60 % done
70 % done
80 % done
90 % done
100 % done


### Creating the submission file

In [11]:
masks_to_submission(submission_path, *filenames)
print('Submission file created, saved at', submission_path)

Submission file created, saved at ../submission.csv


### Applying post-processing to the images

In [12]:
if not os.path.isdir(postprocess_path):
    os.mkdir(postprocess_path)

post_processed_list = []
for i in range(1,TESTING_SIZE+1): 
    p_img = get_pred_postprocessed(pred_test_path, i, 'test',IMG_PATCH_SIZE)
    filename = postprocess_path + "gt_pred_pp_" + str(i) + ".png"
    p_img = np.asarray(p_img)
    p_img = np.multiply(p_img,255.0)
    p_img = Image.fromarray(p_img)
    post_img = p_img.convert('RGB')
    post_processed_list.append(filename)
    post_img.save(filename)
    
    pred = mpimg.imread(filename)
    imageid = "/test_%d" % i
    image_filename = test_data_filename + imageid + imageid + ".png"
    original_img = mpimg.imread(image_filename)
    
    overlay = make_img_overlay(original_img, pred[:,:,0], PIXEL_DEPTH)
    overlay.save(postprocess_path + "overlay_pp_" + str(i) + ".png")

### Saving submission file after post-processing

In [13]:
masks_to_submission(pp_submission_path, *post_processed_list)
print('Submission file created, saved at', pp_submission_path)

Submission file created, saved at ../submission.csv
